In [1]:
!pip install ucimlrepo

In [2]:
from ucimlrepo import fetch_ucirepo

# fetch dataset
dry_bean = fetch_ucirepo(id=602)

# data (as pandas dataframes)
X = dry_bean.data.features
y = dry_bean.data.targets

# metadata
print(dry_bean.metadata)

# variable information
print(dry_bean.variables)

{'uci_id': 602, 'name': 'Dry Bean', 'repository_url': 'https://archive.ics.uci.edu/dataset/602/dry+bean+dataset', 'data_url': 'https://archive.ics.uci.edu/static/public/602/data.csv', 'abstract': 'Images of 13,611 grains of 7 different registered dry beans were taken with a high-resolution camera. A total of 16 features; 12 dimensions and 4 shape forms, were obtained from the grains.', 'area': 'Biology', 'tasks': ['Classification'], 'characteristics': ['Multivariate'], 'num_instances': 13611, 'num_features': 16, 'feature_types': ['Integer', 'Real'], 'demographics': [], 'target_col': ['Class'], 'index_col': None, 'has_missing_values': 'no', 'missing_values_symbol': None, 'year_of_dataset_creation': 2020, 'last_updated': 'Thu Mar 28 2024', 'dataset_doi': '10.24432/C50S4B', 'creators': [], 'intro_paper': {'title': 'Multiclass classification of dry beans using computer vision and machine learning techniques', 'authors': 'M. Koklu, Ilker Ali Özkan', 'published_in': 'Computers and Electronic

In [18]:
import pandas as pd
import numpy as np
import random

In [4]:
y['Class'].unique()

array(['SEKER', 'BARBUNYA', 'BOMBAY', 'CALI', 'HOROZ', 'SIRA', 'DERMASON'],
      dtype=object)

In [5]:
y['Class'].replace(to_replace = ['SEKER', 'BARBUNYA', 'BOMBAY', 'CALI', 'HOROZ', 'SIRA', 'DERMASON'],
                   value = [i for i in range(7)],
                   inplace = True)

<ipython-input-5-17ae000170a6>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y['Class'].replace(to_replace = ['SEKER', 'BARBUNYA', 'BOMBAY', 'CALI', 'HOROZ', 'SIRA', 'DERMASON'],


In [6]:
y['Class']

0        0
1        0
2        0
3        0
4        0
        ..
13606    6
13607    6
13608    6
13609    6
13610    6
Name: Class, Length: 13611, dtype: int64

In [7]:
X = np.array(X)
y = np.array(y)

In [8]:
p = np.arange(X.shape[0])
np.random.shuffle(p)
X = X[p]
y = y[p]

In [9]:
X = (X-X.mean(axis=0))/X.std(axis=0)

In [10]:
X_train = X[:int(.8*X.shape[0])]
X_test = X[int(.8*X.shape[0]):]
y_train = y[:int(.8*X.shape[0])]
y_test = y[int(.8*X.shape[0]):]

In [77]:
N = X_train.shape[0]
rangeN = range(N)
D = 16
K = 7
H1 = 128#size of first hidden layer
W1 = np.random.rand(D,H1)
b1 = np.zeros((1,H1))
W2 = np.random.rand(H1,K)
b2 = np.zeros((1,K))

In [78]:
num_iters = 50000
lr = 1e-3
reg = 1e-3
batch_size = 1

In [59]:
def sigmoid(z):
  return 1/(1+np.exp(-z))

def softmax(z):
  m = np.max(z, axis = 1, keepdims = True)
  e_z = np.exp(z-m)
  return e_z/np.sum(e_z, axis = 1, keepdims = True)

In [79]:
for i in range(num_iters):
  batch = random.sample(rangeN, batch_size)
  X_b = X_train[batch]
  y_b = y_train[batch]

  hidden = sigmoid(X_b.dot(W1)+b1)
  scores = hidden.dot(W2)+b2
  probs = softmax(scores)

  data_loss = -np.mean(np.log(probs[range(batch_size), y_b]))
  reg_loss = 0.5*reg*np.sum(W1*W1) + 0.5*reg*np.sum(W2*W2)
  loss = data_loss + reg_loss

  if i%100 == 0:
    print(f'{i}th iteration loss {loss}')

  dscores = probs
  dscores[range(batch_size), y_b] -= 1
  dscores /= batch_size
  dW2 = (hidden.T).dot(dscores)
  db2 = dscores[0]
  dhidden = dscores.dot(W2.T)
  temp = dhidden*hidden*(1-hidden)
  dW1 = (X_b.T).dot(temp)
  db1 = temp[0]

  dW1 += reg*W1
  dW2 += reg*W2

  W1 -= lr*dW1
  b1 -= lr*db1
  W2 -= lr*dW2
  b2 -= lr*db2

0th iteration loss 2.385806208370454
100th iteration loss 1.3877308773190227
200th iteration loss 2.1825226654144667
300th iteration loss 1.8450282195764331
400th iteration loss 1.8384591896542894
500th iteration loss 2.299718910283829
600th iteration loss 3.1427739809868633
700th iteration loss 2.0339807951553257
800th iteration loss 1.950378516601675
900th iteration loss 1.631817940163582
1000th iteration loss 1.9300943262100025
1100th iteration loss 3.102635903224755
1200th iteration loss 2.4420720386907253
1300th iteration loss 2.2001978494797245
1400th iteration loss 2.202066719696002
1500th iteration loss 1.2161230475880427
1600th iteration loss 1.5417999257187547
1700th iteration loss 1.569459823222764
1800th iteration loss 1.2559768642597158
1900th iteration loss 1.3168180585027536
2000th iteration loss 2.5454457388266696
2100th iteration loss 0.9067663924270862
2200th iteration loss 2.050874561717598
2300th iteration loss 1.7552118240311907
2400th iteration loss 0.985944294399

In [33]:
def pred(X):
  hidden = sigmoid(X.dot(W1)+b1)
  scores = hidden.dot(W2)+b2
  probs = softmax(scores)
  return np.argmax(probs, axis = 1, keepdims = True)

In [80]:
y_pred = pred(X_test)

In [35]:
y_pred.shape

(2723, 1)

In [81]:
correct = 0
for i in range(len(X_test)):
  if y_pred[i] == y_test[i]:
    correct += 1
print(100*correct/len(X_test))

90.12118986412045
